## Module 2: Mineral-Machine Learning (MIN-ML)

In [ ]:
""" Created on June 30, 2023 // @author: Sarah Shi """

### I. Introduction

#### Motivations for Applying Machine Learning to Mineral Data 

Explorations of mineral compositions aiming to reveal complex magmatic processes in melts have proliferated with the growing accessibility of geochemical datasets through databases including PetDB, LEPR/TraceDs, and GEOROC and of computational methods [1]. The generation and continuous quality assurance of mineral data in these databases requires significant human intervention and individual post-processing. One major problem is that minerals may be misclassified (i.e., a compiled dataset of clinopyroxenes may contain some amphiboles), and compilations may contain poor-quality electron microprobe (EPMA) analyses (with low totals, low cation sums, or poor correspondence to the theoretical stoichiometry of a mineral phase). At the moment, individual studies compiling geochemical datasets for specific tectonic settings [2] or calibrating thermobarometers based on mineral-melt equilibrium [3] tend to apply their own filters. With a push for a more consistent approach, we create a new open-source Python package called MIN-ML (MINeral classification using Machine Learning) for classifying common igneous minerals based on oxide data collected by EPMA, with functions for calculating stoichiometries and crystallographic sites based on this classification. Utilizing this package allows for the identification of misclassified mineral phases and poor-quality data. We streamline data processing and cleaning to allow for the rapid transition to usable data, improving the utility of data curated in these databases and furthering computing and modeling capabilities. 

While mineral identification and classification are obviously critical to the success of computational methodologies and machine learning (ML) applied to these large datasets, the question of how to best classify minerals from EPMA analyses comes to the fore. We approach this question by exploring and developing ML workflows, both supervised (classification algorithms) and unsupervised (dimensionality reduction and clustering). Unsupervised methods including autoencoders, a type of artificial neural network, present the opportunity to classify minerals with little a priori information. Autoencoders pair two neural networks with an encoder, compressing input data to a dimensionality-reduced latent representation, and a decoder, expanding latent representations to reconstruct the input and minimize loss. We present a novel autoencoder model aimed at meaningfully representing EPMA analyses of minerals in latent space, investigating the relationships between mineral phases, and performing classifications of these minerals. The model is trained with newly compiled datasets of twelve igneous mineral phases on thousands to tens of thousands of analyses per phase – across tectonic settings to train these ML models. The autoencoder is applied to datasets of mineral analyses from PetDB, LEPR, and GEOROC to evaluate model performance and show significant improvements in mineral phase segregation and classification, critical to rigorous dataset quality control and future integration into data processing routines. 

[1] Lehnert, K. A., et al., 2022, IEDA2: Evolving EarthChem, LEPR/TraceDs, and SESAR into a Next Generation Data Infrastructure for Data-Driven Research Paradigms in Geochemistry, Petrology, and Volcanology, in 2022 Goldschmidt Conference.

[2] Gale, A., et al., The mean composition of ocean ridge basalts. Geochemistry, Geophysics, Geosystems 14, 489-518 (2013).

[3] Petrelli, M., et al., Machine learning thermobarometry: Application to clinopyroxene-bearing magmas. JGR: Solid Earth 125, e2020JB020130 (2020).



### II. Accessing GEOROC mineral data with simple coding. 

### What am I looking at here? ###
This is a Jupyter Notebook, a document in a web browser that you can edit, generate text, and run code (typically Python). In this particular case, to help the code run smoothly, we are doing the processing in the cloud using a tool called Google Colab. This means that you should be able to access all of the packages required and execute the code independent of the details of the computer that you are using locally - all you need is access to an internet connection and a web browser.


### Online Big Data - Mineral compositions ###
A key feature of modern earth and environmental sciences is that huge observational, experimental and thermodynamic datasets are now available. The IEDA2 data infrastructures hosts observational datasets with EarthChem and experimental and thermodynamic datasets with LEPR/TraceDs (Library of Experimental Phase Relations/Trace element Distribution experimental database). The GEOROC databases similarly hosts large datasets of petrologic data. Tools like Python allow for increased interaction with these datasets. In this practical we will look at a GEOROC dataset of electron microprobe analyses of minerals, which you can find on [GEOROC](https://doi.org/10.25625/SGFTFN). I take a subset of the full 856k dataset (20%) to make the data a bit more tractable. You can click on the links to understand the data sources. 

We focus on igneous minerals for this study, and thus we limit the mineral analyses to: 

- Amphibole
- Apatite
- Biotite
- Clinopyroxene
- Garnet
- Ilmenite
- K-Feldspar
- Magnetite
- Muscovite
- Olivine 
- Orthopyroxene
- Plagioclase
- Quartz
- Rutile
- Spinel 
- Tourmaline 
- Zircon 

### NumPy for simple maths, Pandas for data-tables ###
We will explore simple chemical relationships in the data using [numpy](https://numpy.org/) to perform simple mathematical calculations and using [pandas](https://pandas.pydata.org/) to read in data. We will use [matplotlib](https://matplotlib.org/) and [seaborn](https://seaborn.pydata.org/index.html) to plot. 

I'll put a couple of cells of code in below, which you should run, in order to import the correct packages and bathymetry data for use in the practical. 


In [ ]:
# Import standard Python packages for math, data, and plotting. 

import numpy as np
import pandas as pd
import seaborn as sns

from matplotlib import pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

from matplotlib import rc
rc('font',**{'size': 14})

### III. Analyze the data. 

##### 1. Open the file you have just downloaded in Excel. Notice the huge amount of data! This is only a subset of the mineral analyses available on GEOROC. Petrologists have definitely been busy!

In [ ]:
# Read in the datasets here with the Pandas package. We are storing the data as a dataframe, which is a 2D labeled data structure, like a 2D array or table with rows and columns. 

df = pd.read_csv("https://raw.githubusercontent.com/sarahshi/earthchem-teaching/main/data/GEOROC_subset.csv")

df

In [ ]:
# Visualize the dataset by calling the first 5 rows of data. What information is available?

df.head()


In [ ]:
# We can print all column names to understand what is available by calling: 

df.columns


In [ ]:
# You can access particular columns of data by referencing them as follows: 

df['SiO2']


Note the presence of NaN (Not a Number) values. These are simply blank values in the Excel file. 

##### 2. Looking at df.head() gives us an indication of the columns present, including the one called 'Mineral', indicating the published mineral class. Let's further understand and visualize the data for each mineral class. 

In [ ]:
# We can print the information stored in the 'Mineral' column, by calling: 

df['Mineral']


In [ ]:
# We only see a few of all available mineral classes, but we can also display all the unique values within the column, by calling: 

df['Mineral'].unique()


In [ ]:
# What if we only want to look at the olivine data? We can use Python to subselect a portion of the data as follows: 

ol_df = df[df['Mineral'] == 'Olivine']

ol_df


In [ ]:
# Let's plot up these oxide distributions with a violin plot. 

oxides = ['SiO2', 'TiO2', 'Al2O3', 'FeOt', 'MnO', 'MgO', 'CaO', 'Na2O', 'K2O', 'Cr2O3']

fig, ax = plt.subplots(1, 1, figsize=(5, 5))
ax.violinplot(ol_df[oxides].fillna(0), positions = np.linspace(0, 9, 10), showmeans = True, showextrema= False)
ax.set_title('Olivine')
ax.set_xlabel('Oxide')
ax.set_ylabel('Weight Percentage')
ax.set_xticks(np.linspace(0, 9, 10))
ax.set_xticklabels(oxides, rotation = 45, fontsize = 15)
ax.set_ylim([-5, 105])
plt.tight_layout()
plt.show()


##### 3. Olivine contains primarily $SiO_2$, $FeO$, and $MgO$ as expected. This is great to visualize, but how do we analyze the entire dataset at the same time? What if I want to automate making violin plots for every mineral, instead of subsetting and creating a different figure each time? 


In [ ]:
phase = df['Mineral'].unique()

fig, ax = plt.subplots(3, 5, figsize=(25, 15))
ax = ax.flatten()
for i in range(len(phase)): 
    if (df['Mineral'] == phase[i]).sum() > 0:
        ax[i].violinplot(df[df['Mineral']==phase[i]][oxides].fillna(0), positions = np.linspace(0, 9, 10), showmeans = True, showextrema= False)
        ax[i].set_title(phase[i])
        ax[i].set_xticks(np.linspace(0, 9, 10))
        ax[i].set_xticklabels(oxides, rotation = 45, fontsize = 15)
        ax[i].set_ylim([-5, 105])

fig.supylabel('Weight Percentage')
fig.supxlabel('Oxide')
plt.tight_layout()


### IV. Model the data. 

Our goal is to classify minerals from the geochemistry. We can take a twofold approach: 

- Supervised: We assume prior knowledge of the mineral phase label. We see how algorithms can learn to classify things properly. 
- Unsupervised: We assume no prior knowledge of the mineral phase label. We see what patterns come out of the data. 


##### 1. Supervised Learning with Gaussian Discriminant Analysis

We can appreciate that each mineral has distinct oxide composition distributions, usually normally distributed (potentially with long tails). The two minerals that are the most chemically similar are amphibole and clinopyroxene. The geochemical similarity between amphibole and clinopyroxene is well known, often leading petrologists to mistake the two minerals. 

[Gaussian Discriminant Analysis (GDA)](https://www.eecs189.org/static/notes/n18.pdf) is one way in which multivariate Gaussian distributions of features can be modeled, assuming uniqueness. The link contains some excellent information on the math behind GDA. Let's apply GDA to this dataset! We will do this by hand rather than using a Python package, to better understand the methodology. 


In [ ]:
# Let's pull a portion to serve as our training dataset -- what the model is calibrated upon. 

from sklearn.model_selection import train_test_split

df_train, testing = train_test_split(df, train_size=0.5, shuffle=True, random_state=42)

df_train

# Normally, we would perform a closer analysis of the data in the training dataset, but this module is more about presenting this methodology rather than about the specifics of these data. 


In [ ]:
# We want to analyze the mean and covariance matrices of each mineral class. To do so, let's define a function: 

def mean_cov(df):
    oxides = ['SiO2', 'TiO2', 'Al2O3', 'FeOt', 'MnO', 'MgO', 'CaO', 'Na2O', 'K2O', 'Cr2O3']
    data = df[oxides].values

    # Calculate mean and covariance matrix
    mean = np.mean(data, axis=0)
    cov = np.cov(data, rowvar=False)

    return mean, cov


In [ ]:
# Computing the mean and covariance. The mean_cov function calculates the mean and covariance matrix for a given subset of the dataset, which represents the feature statistics for each class.

amp_mean, amp_cov = mean_cov(df_train[df_train.Mineral=='Amphibole'].fillna(0))
ap_mean, ap_cov = mean_cov(df_train[df_train.Mineral=='Apatite'].fillna(0))
bt_mean, bt_cov = mean_cov(df_train[df_train.Mineral=='Biotite'].fillna(0))
cpx_mean, cpx_cov = mean_cov(df_train[df_train.Mineral=='Clinopyroxene'].fillna(0))
gt_mean, gt_cov = mean_cov(df_train[df_train.Mineral=='Garnet'].fillna(0))
il_mean, il_cov = mean_cov(df_train[df_train.Mineral=='Ilmenite'].fillna(0))
ksp_mean, ksp_cov = mean_cov(df_train[df_train.Mineral=='KFeldspar'].fillna(0))
ms_mean, ms_cov = mean_cov(df_train[df_train.Mineral=='Magnetite'].fillna(0))
ol_mean, ol_cov = mean_cov(df_train[df_train.Mineral=='Olivine'].fillna(0))
opx_mean, opx_cov = mean_cov(df_train[df_train.Mineral=='Orthopyroxene'].fillna(0))
plag_mean, plag_cov = mean_cov(df_train[df_train.Mineral=='Plagioclase'].fillna(0))
qtz_mean, qtz_cov = mean_cov(df_train[df_train.Mineral=='Quartz'].fillna(0))
sp_mean, sp_cov = mean_cov(df_train[df_train.Mineral=='Spinel'].fillna(0))
zr_mean, zr_cov = mean_cov(df_train[df_train.Mineral=='Zircon'].fillna(0))


In [ ]:
# Create multivariate normal distributions: The multivariate_normal function is used to create a multivariate normal distribution for each class. 
# The mean and covariance matrix obtained in the previous step are used as parameters for these distributions.

from scipy.stats import multivariate_normal

amp_rv = multivariate_normal(amp_mean, amp_cov, allow_singular=True)
ap_rv = multivariate_normal(ap_mean, ap_cov, allow_singular=True)
bt_rv = multivariate_normal(bt_mean, bt_cov, allow_singular=True)
cpx_rv = multivariate_normal(cpx_mean, cpx_cov, allow_singular=True)
gt_rv = multivariate_normal(gt_mean, gt_cov, allow_singular=True)
il_rv = multivariate_normal(il_mean, il_cov, allow_singular=True)
ksp_rv = multivariate_normal(ksp_mean, ksp_cov, allow_singular=True)
ms_rv = multivariate_normal(ms_mean, ms_cov, allow_singular=True)
ol_rv = multivariate_normal(ol_mean, ol_cov, allow_singular=True)
opx_rv = multivariate_normal(opx_mean, opx_cov, allow_singular=True)
plag_rv = multivariate_normal(plag_mean, plag_cov, allow_singular=True)
qtz_rv = multivariate_normal(qtz_mean, qtz_cov, allow_singular=True)
sp_rv = multivariate_normal(sp_mean, sp_cov, allow_singular=True)
zr_rv = multivariate_normal(zr_mean, zr_cov, allow_singular=True)


In [ ]:
# Determining the best fit mineral: The class with the highest probability is selected as the predicted class for the data instance.

# Store all the random variables in a dictionary for easier access
rv_dict = {'Amphibole': amp_rv, 'Apatite': ap_rv, 'Biotite': bt_rv, 'Clinopyroxene': cpx_rv,
           'Garnet': gt_rv, 'Ilmenite': il_rv, 'KFeldspar': ksp_rv, 'Magnetite': ms_rv,
           'Olivine': ol_rv, 'Orthopyroxene': opx_rv, 'Plagioclase': plag_rv, 'Quartz': qtz_rv,
           'Spinel': sp_rv, 'Zircon': zr_rv}

# Define a function that will be applied to each row
def get_best_fit_mineral(row):
    # Calculate the PDF for each distribution and store the results in a dictionary
    probs = {mineral: rv.pdf(row) for mineral, rv in rv_dict.items()}
    
    # Find the mineral with the highest probability
    best_fit_mineral = max(probs, key=probs.get)

    return best_fit_mineral


In [ ]:
# We now want to apply this function to a test dataset. We pull some GEOROC data from the Cascades to perform this test. 

df_test = pd.read_csv("https://raw.githubusercontent.com/sarahshi/earthchem-teaching/main/CpxAmp_April23.csv")

# Use the `apply` method to apply the function to each row

df_test['Best-Fit Mineral'] = df_test[oxides].fillna(0).apply(get_best_fit_mineral, axis=1)
df_test


In [ ]:
# Let's check which values in the dataframe have published and predicted classifications agreeing vs. disagreeing. 

true = df_test[df_test['Mineral']==df_test['Best-Fit Mineral']]
false = df_test[df_test['Mineral']!=df_test['Best-Fit Mineral']]

print(str(len(true)) + ' classifications agree and ' + str(len(false)) + ' classifications disagree')


In [ ]:
true

In [ ]:
false

We can focus on minerals with common misclassifications -- in this case, clinopyroxene and amphibole. 

In [ ]:

pubcpx_predcpx = true[(true['Mineral']=='Clinopyroxene') & (true['Best-Fit Mineral']=='Clinopyroxene')]

pubcpx_predamp = false[(false['Mineral']=='Clinopyroxene') & (false['Best-Fit Mineral']=='Amphibole')]

pubcpx_predamp


In [ ]:

pubamp_predamp = true[(true['Mineral']=='Amphibole') & (true['Best-Fit Mineral']=='Amphibole')]

pubamp_predcpx = false[(false['Mineral']=='Amphibole') & (false['Best-Fit Mineral']=='Clinopyroxene')]

pubamp_predcpx


In [ ]:

plt.figure(figsize=(8, 8))
plt.scatter(pubcpx_predcpx['SiO2'], pubcpx_predcpx['Na2O'], color='tab:red', linewidth=0.2, ec='k', alpha=0.7, label='Pub:Cpx, GDA:Cpx')
plt.scatter(pubamp_predamp['SiO2'], pubamp_predamp['Na2O'], color='tab:blue', linewidth=0.2, ec='k', alpha=0.7, label='Pub:Amp, GDA:Amp')

plt.scatter(pubcpx_predamp['SiO2'], pubcpx_predamp['Na2O'], color='cyan', marker='d', s=60, linewidth=0.2, ec='k', label='Pub:Cpx, GDA:Amp')
plt.scatter(pubamp_predcpx['SiO2'], pubamp_predcpx['Na2O'], color='yellow', marker='d', s=60, linewidth=0.2, ec='k', label='Pub:Amp, GDA:Cpx')

plt.xlim([35, 60])
plt.ylim([-0.1, 4.1])
plt.xlabel('SiO2 (wt.%)')
plt.ylabel('Na2O (wt.%)')
plt.legend()
plt.tight_layout()

##### A. How do predictions from GD compare to predictions from the neural network of the MIN-ML talk? See the results in the following figure:

![minml_class](https://raw.githubusercontent.com/sarahshi/earthchem-teaching/main/figures/minml_class.png)


##### 2. Unsupervised Learning with PCA

Principal Components Analysis (PCA) is a linear transformation that resolves high dimensionality data and constructs principle components, or lower dimensionality representations of interrelated data. PCA constructs a covariance (and sometimes, the correlation) matrix of data and solves this eigenvector/eigenvalue problem to determine the matrix eigenvectors. The principle components, or the eigenvectors corresponding to the largest eigenvalues, thus identify the directions along which the variation in data is maximized in n-dimensional space.

We can apply methods like PCA to glean what patterns exist within the dataset. 


In [ ]:
import time
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

oxides = ['SiO2', 'TiO2', 'Al2O3', 'FeOt', 'MnO', 'MgO', 'CaO', 'Na2O', 'K2O', 'Cr2O3']
label = ['Mineral']

min = df_train[label]
wt = df_train[oxides].fillna(0).to_numpy()
# Here, we apply a StandardScaler or z-score normalization to ensure that the oxides have similar numeric ranges. 
wt_scale = StandardScaler().fit_transform(wt)

pca = PCA(n_components = 8)
pca_arr = pca.fit_transform(wt_scale)

print(str(round(pca.explained_variance_ratio_.sum()*100, 2)) + '% of the variability in data is explained by 8 principal components')

In [ ]:
# Let's plot up our PCA results for each mineral class. This code plots the first three principal components and automates the plotting process.  

import matplotlib.colors as mcolors
import matplotlib.cm as mcm

phase = df['Mineral'].unique()
phasez = range(1,len(phase))

tab = plt.get_cmap('tab20')
cNorm  = mcolors.Normalize(vmin=0, vmax=len(phase))
scalarMap = mcm.ScalarMappable(norm=cNorm, cmap=tab)

fig, ax = plt.subplots(1, 3, figsize = (21, 7))
ax = ax.flatten()
# My feature_normalisation function has the same function as normalization from sklearn.preprocessing
for i in range(len(phase)):
    indx = min['Mineral'] == phase[i]
    ax[0].scatter(pca_arr[indx][:, 0], pca_arr[indx][:, 1], s=15, color=scalarMap.to_rgba(i), lw=0.1, ec='k', alpha=0.75, label=phase[i])
    ax[1].scatter(pca_arr[indx][:, 0], pca_arr[indx][:, 2], s=15, color=scalarMap.to_rgba(i), lw=0.1, ec='k', alpha=0.75, label=phase[i])
    ax[2].scatter(pca_arr[indx][:, 1], pca_arr[indx][:, 2], s=15, color=scalarMap.to_rgba(i), lw=0.1, ec='k', alpha=0.75, label=phase[i])
ax[0].legend(prop={'size': 8})
ax[0].set_title('Normalization wt% - PCA')
ax[0].set_xlabel('PC1')
ax[0].set_ylabel('PC2')

ax[1].legend(prop={'size': 8})
ax[1].set_title('Normalization wt% - PCA')
ax[1].set_xlabel('PC1')
ax[1].set_ylabel('PC3')

ax[2].legend(prop={'size': 8})
ax[2].set_title('Normalization wt% - PCA')
ax[2].set_xlabel('PC2')
ax[2].set_ylabel('PC3')

plt.tight_layout()
plt.show()

##### A. How do predictions from PCA compare to predictions from the autoencoder of the MIN-ML talk? See the results in the following figure:

![minml_autoencoder](https://raw.githubusercontent.com/sarahshi/earthchem-teaching/main/figures/minml_autoencoder.png)
